In [103]:
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from math import ceil
from datetime import timedelta
import unicodedata  

In [104]:
#0 == False
#1 == True

in_colab = 1
if in_colab == True:
    !unzip Dataset.zip  
    ! mv /content/Dataset/* /content/

Archive:  Dataset.zip
 extracting: Dataset/Controle de NC 2022.xlsx  
 extracting: Dataset/Proponentes.xlsx  


In [105]:
start_time = time()

excel_file1 = 'Controle de NC 2022.xlsx'
lst_sheets = ['SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreção']
excel_file2 = 'Proponentes.xlsx'
lst_sheets.append('Proponentes')

#----------------------------------------------------------

df_sn  = pd.read_excel(excel_file1, sheet_name=lst_sheets[0],usecols="B:J", skiprows=3,parse_dates=True)

df_an = pd.read_excel(excel_file1, sheet_name=lst_sheets[1],usecols="B:J", skiprows=1,parse_dates=True)
#skiprows=1,
#usecols="B:J") 
#
df_ic = pd.read_excel(excel_file1, sheet_name='ÍndiceCorreção',
skiprows=1,
usecols="B:E")

df_pr= pd.read_excel(excel_file2, sheet_name='Proponentes')
##----------------------------------------------------------

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time


Time: This code took 1 seconds to run.


In [106]:
start_time = time()

all_dfs = [df_sn, df_an, df_ic, df_pr]

def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(cln.strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    return df

for df in all_dfs:
    rename_df(df)

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time



Time: This code took 1 seconds to run.


In [107]:
start_time = time()
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings
def regexing(s):
    lst = [
    ('PRF ','POLICIA RODOVIARIA FEDERAL '),
    ('DEP. FEDERAL ', ''),
    ('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
    ('DEP. FEDERAL', ''),
    ('PRF ','POLICIA RODOVIARIA FEDERAL '),
    ('MIN. ','MINISTERIO '),
    ('SENADORA ',''),
    ('SENADOR ',''),
    ]
    for string in lst:
        return re.sub(string[0],string[1],s)



lst_proponente = []

for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        lst_proponente.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        lst_proponente.append('DEP. ' + prop)
    else: 
        lst_proponente.append(prop)

df_pr['prop_pk2'] = lst_proponente

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This code took 1 seconds to run.

Time: This code took 1 seconds to run.


In [108]:
start_time = time()

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an]
for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    df['proponente'] = df['proponente'].apply(lambda x:str(x).strip())
#
lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:str(x).strip())
for cln in lst_pr:
    cln = cln.apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [109]:
start_time = time()

#criando lista dos DataFrames pelo qual o loop passara 
lst_df = (df_sn, df_an)

#criando as colunas das primary_keys nos 3 dataframes
for df in lst_df:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''
    
#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x == lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2= []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 2 seconds to run.


In [110]:
#cleaning analitic sintetic rows
df_sn = df_sn.loc[df_sn['proponente'] != 'SUBTOTAL']  
df_sn = df_sn.loc[df_sn['proponente'] != 'TOTAL']
df_sn = df_sn.loc[df_sn['proponente'] != 'NAN']
df_sn.fillna(0,inplace=True)
df_sn.iloc[-5:]

,nc,proponente,programa,empenho,valor_inicial_nc,valor_utilizado,valor_economizado,saldo,consumo_da_nc_%,prop_pk1,prop_pk2
73,73/2021,ULDURICO JUNIOR,WifiBrasil,2021NE000186,141000.0,40000.0,10054.977200,101000.0,28.368794,ULDURICO ALENCAR PINTO,DEP. ULDURICO ALENCAR PINTO
74,74/2021,SORAYA THRONICKE,WifiBrasil,2021NE000193,155000.0,20000.0,12620.344215,135000.0,12.903226,SORAYA THRONICKE,SEN. SORAYA THRONICKE
75,75/2021,MCOM,WifiBrasil,2021NE000065,500000.0,60000.0,10416.976070,440000.0,12.000000,MINISTERIO DAS COMUNICACOES,MINISTERIO DAS COMUNICACOES
76,76/2021,CABO JUNIO AMARAL,WifiBrasil,2021NE000123,93000.0,70000.0,44982.509309,23000.0,75.268817,GERALDO JUNIO DO AMARAL,DEP. GERALDO JUNIO DO AMARAL
77,77/2021,LEUR LOMANTO,WifiBrasil,2021NE000100,28000.0,30000.0,6934.680205,-2000.0,107.142857,LEUR ANTONIO DE BRITTO LOMANTO JUNIOR,DEP. LEUR ANTONIO DE BRITTO LOMANTO JUNIOR


In [111]:
#checando se tem dfs sem prop_pk 
start_time = time()

lst  = [df_sn, df_an]
lst_no_cv_name = []
for x in lst:
    for index, value in enumerate(df.loc[df.prop_pk1==''].proponente.values):
        if value not in lst_no_cv_name:
            lst_no_cv_name.append(df.loc[df.prop_pk1==''].proponente.values[index])
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_pr_nokey = pd.DataFrame()
df_pr_nokey['prop_nokey']= lst_no_cv_name
df_pr_nokey['prop_pk1']= ''
df_pr_nokey['orgao']=''
df_pr_nokey['uf']=''
df_pr_nokey['partido']=''


df_pr_nokey.to_csv('ignore_prop_with_no_key.csv')
df_pr_nokey

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
Time: This code took 1 seconds to run.


In [112]:
start_time = time()

!pip install xlsxwriter

dfs = (df_sn, df_an, df_ic, df_pr)
dfs[0].name = lst_sheets[0]
dfs[1].name = lst_sheets[1]
dfs[2].name = lst_sheets[2]
dfs[3].name = lst_sheets[3]

writer = pd.ExcelWriter('BI_Financeiro.xlsx',engine='xlsxwriter')
workbook=writer.book

for df in dfs:
  worksheet=workbook.add_worksheet(df.name)
  writer.sheets[df.name] = worksheet
  df.to_excel(writer,sheet_name=df.name,startrow=0 , startcol=0, index=False)

writer.save()

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This code took 1 seconds to run.

Time: This code took 4 seconds to run.


In [113]:
final_time= str(timedelta(seconds= total_time)); print(f'Time: The total time to run all the notebook was {final_time}.')
#Time: The total time to run all the notebook is 00:39 minutes.

Time: The total time to run all the notebook was 0:00:05.302143.
